In [ ]:
import pandas as pd
import json
pd.options.display.max_rows = 999

# Create new_countries.json

In [ ]:
countries = open('../frontend/public/unep-gpml.topo.json')
countries = json.loads(countries.read())

In [ ]:
countries = [x['properties'] for x in countries['objects']['Country_Polygon']['geometries']]
countries = pd.DataFrame(countries)

In [ ]:
countries

In [ ]:
countries.groupby('ROMNAM').count()['OBJECTID'].nlargest(10)

In [ ]:
countries[countries['M49Code'].isna()]

In [ ]:
countries[countries['ISO3CD'] == 'SHN']

In [ ]:
countries[countries['ROMNAM'] != countries['MAP_LABEL']]

In [ ]:
countries[countries['ISO3CD'] == 'SHN'].to_dict('records')

In [ ]:
countries.groupby('M49Code').count()['OBJECTID'].nlargest(5)

In [ ]:
## Rename ROMNAM for SHN ISO3CD
def renameRomnam(df):
    if (df['ISO3CD'] == 'SHN'):
        return df['MAP_LABEL'].replace(" *", "")
    return df['ROMNAM']

countries['ROMNAM'] = countries.apply(renameRomnam, axis=1)

In [ ]:
countries = countries.rename(columns={'M49Code':'id','ISO3CD':'iso_code','ROMNAM':'name', 'MAP_LABEL':'label', 'MAP_COLOR':'territory','STATUS':'description'})
countries = countries[['id','iso_code','label', 'name', 'territory','description']]
countries['member'] = countries['description'].apply(lambda x: 1 if x == 'Member State' else 2)

In [ ]:
## FILTER WHERE ISOCODE IS NOT NULL (remove lake value)
countries = countries[countries['iso_code'].notna()]

In [ ]:
## DROP DUPLICATE
countries = countries.sort_values(['member','iso_code']).drop_duplicates(subset=['iso_code', 'name', 'label'], keep='first')
countries = countries.drop(columns=['member'])

In [ ]:
## FILTER VALUE WHERE ID IS NOT NULL
countries = countries[countries['id'].notnull()]

In [ ]:
## FILTER VALUE WHERE NAME IS NOT NULL
countries = countries[countries['name'].notnull()]

In [ ]:
## CHANGE ID TO INT
countries['id'] = countries['id'].fillna(0)
countries['id'] = countries['id'].astype(int)

In [ ]:
countries = countries.sort_values('id').reset_index(drop=True)

In [ ]:
countries.sort_values('id').drop(columns=['label']).to_json('../backend/dev/resources/files/new_countries.json', orient='records', indent=2)

# Map countries old id into new id `{old : new}`

In [ ]:
old = open('../backend/dev/resources/files/countries.json')
old = json.loads(old.read())
old = pd.DataFrame(old)

In [ ]:
old = old.sort_values('iso_code')
countries = countries.sort_values('iso_code')

In [ ]:
countries['name'] = countries['label'].fillna(" ")

In [ ]:
mapping = {};

In [ ]:
def mappingId(df):
    new_name = df['label']
    if new_name is None:
        new_name = ""
    if "*" in new_name:
        new_name = new_name.split(" *")[0]
    if "North" in new_name:
        new_name = new_name.split("North ")
        new_name = new_name[0]
        if len(new_name) > 0:
            new_name = new_name[1]
    if "Canary" in new_name:
        new_name = new_name.split(" ")[0]
    if "," in new_name:
        new_name = new_name.split(", ")[0]
    if new_name == 'Heard Is. & McDonald Is. (Aust.)':
        new_name = 'Heard Isl. & McDonald Is. (Aust.)'
    if new_name == 'Norfolk Island (Aust.)':
        new_name = 'Norfolk Is. (Aust.)'
    if new_name == 'Azores Islands (Port.)':
        new_name = 'Azores Is. (Port.)'
    if new_name == 'Clipperton Island':
        new_name = 'Clipperton Is. (Fr.)'
    if new_name == 'Jarvis Island (USA)':
        new_name = 'Jarvis Is. (USA)'
    if new_name == 'Howland Island (USA)':
        new_name = 'Howland Is. (USA)'
    if new_name == 'Tajikistan':
        new_name = 'Tadjikistan'   
    if df['name'] == 'Chagos Archipelago (Mauritius)':
        df['name'] = 'Chagos Archipelagio **'
            
    oldData = old[
        ((old['iso_code'] == df['iso_code']) & (old['name'].str.contains(new_name))) 
        | ((old['iso_code'] == df['iso_code']) & (old['name'] == new_name))
        | ((old['iso_code'] == df['iso_code']) & (old['name'] == df['name']))
        | ((old['iso_code'] == df['iso_code']) & (old['name'] == df['label']))
        | (old['name'] == df['name'])
    ]
    
    oldId = oldData['id'].values
    if len(oldId) > 0:
        mapping.update({str(oldId[0]): df['id']})
        return "{0}, {1}".format(oldId[0], df['id'])
    mapping.update({str(df['id']): df['id']})
    return "{0}, {1}".format(df['id'], df['id'])

In [ ]:
countries['mapping'] = countries.apply(mappingId, axis=1)

In [ ]:
with open('../backend/dev/resources/files/new_countries_mapping.json', 'w') as outfile:
    json.dump(mapping, outfile)

In [ ]:
countries[countries['id'] == 1101]

In [ ]:
old[old['id'] == 205]